## EXAMPLE SIMULATIONS

These are example simulations to test out the functioning of the simulation library

In [ ]:
# IMPORT MODULES

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from src.ringtrap_python.molecular_dynamics import sim_leapfrog
from src.ringtrap_python.potentials import LocalHarmonicPotential1D, InverseSquarePotential1D, MutualCoulombPotential, LinearQuadrupolePseudoPotential, testPseudoPot
from src.ringtrap_python.initializers import fibonacciSphere, genMaxwellVelocities, steepestDescent

In [ ]:
%matplotlib widget
mpl.rc('xtick',labelsize=16)
mpl.rc('ytick',labelsize=16)

### 1 ATOM 1D HARMONIC POTENTIAL

Here we implement a single atom in a 1D harmonic potential and compare with theory.
For a 1D harmonic oscillator, $\omega = \sqrt{\frac{k}{m}}$.

So for a particle of mass 2kg and spring constant 10N/m, $\omega = \sqrt{5}$.
If the particle started off at $X_0 = 3.5$ m with 0 velocity then the particle should oscillate with a mixmum amplitude of 3.5m

In [ ]:
# ENSEMBLE DEFINITIONS
"""
n       : The number of particles in the simulation
charge  : Charge of the particle in C
mass    : Mass of the particle in kg
"""
ensemble_prop = {
    "n" : 1,
    "charge" : np.array([1]),
    "mass": np.array([2])
}

n = ensemble_prop["n"]

# Angular frequency
w = np.array([np.sqrt(5.0)])

# Initial position
x0 = np.array([3.5])

# Initial velocity
v0 = np.array([0.0])

In [ ]:
# Creates local 1D harmonic potential around origin with frequency w
potentials = [LocalHarmonicPotential1D(w)]

In [ ]:
# SIMULATION SETUP

# Total time
T = 2.9

# Time step
dt = 1e-3

x,v,a = sim_leapfrog(T,dt,x0,v0,ensemble_prop,potentials,language='c++',dims=1)
t = np.linspace(0,T,int(T/dt)+1)

In [ ]:
print(potentials[0].potential(x[:,0],ensemble_prop))

In [ ]:
# PLOTTING
plt.figure(figsize=(13.33,7.5),dpi=96)

plt.grid()
plt.plot(t,x,label='particle 1',linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Position (m)',fontsize=18)
plt.legend(fontsize=18)
plt.rcParams['figure.figsize'] = [10/2.54, 8/2.54]
plt.show()

In [ ]:
plt.figure(figsize=(13.33,7.5),dpi=96)

plt.subplot(221)
plt.grid()
plt.plot(t,x,label='particle 1',linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Position (m)',fontsize=18)
plt.legend(fontsize=16)

plt.subplot(222)
plt.grid()
plt.plot(t,v,linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Speed (m/s)',fontsize=18)

plt.subplot(223)
plt.grid()
plt.plot(t,a,linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Acceleration (m/s^2)',fontsize=18)

plt.subplot(224)
plt.grid()
plt.plot(x[:,0],[potentials[0].potential(i,ensemble_prop) for i in x],linewidth=2)
plt.xlabel('Position (m)',fontsize=18)
plt.ylabel('Potential energy (J)',fontsize=18)

plt.tight_layout()
plt.show()

### 2 DIFFERENT ATOMS IN A 1D HARMONIC POTENTIAL

Consider 2 particles of different masses trapped in the same harmonic poential starting from the same extreme point. Let the mass of the 2nd particle be 1/3rd the mass of the 1st

In [ ]:
# ENSEMBLE DEFINITIONS
"""
n       : The number of particles in the simulation
charge  : Charge of the particle in C
mass    : Mass of the particle in kg
"""
ensemble_prop = {
    "n" : 2,
    "charge" : np.array([1.0,1.0]),
    "mass": np.array([2.0,2.0/3])
}

n = ensemble_prop["n"]

# Angular frequency
w = np.array([np.sqrt(5.0) , np.sqrt(15.0)])

# Initial position
x0 = np.array([3.5 , 3.5])

# Initial velocity
v0 = np.array([0.0, 0.0])

In [ ]:
# Creates local 1D harmonic potential around origin with frequency w
potentials = [LocalHarmonicPotential1D(w)]

In [ ]:
print(x)

In [ ]:
# SIMULATION SETUP

# Total time
T = 2.9

# Time step
dt = 1e-3

x,v,a = sim_leapfrog(T,dt,x0,v0,ensemble_prop,potentials,language='c++',dims=1)
t = np.linspace(0,T,int(T/dt)+1)

In [ ]:
# PLOTTING
plt.figure(figsize=(13.33,7.5),dpi=96)

plt.grid()
for i in range(n):
    plt.plot(t,x[:,i],label='particle '+str(i+1),linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Position (m)',fontsize=18)
plt.legend(fontsize=18)
plt.rcParams['figure.figsize'] = [10/2.54, 8/2.54]
plt.show()

In [ ]:
plt.figure(figsize=(13.33,7.5),dpi=96)

plt.subplot(221)
plt.grid()
for i in range(n):
    plt.plot(t,x[:,i],label='particle '+str(i + 1),linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Position (m)',fontsize=18)
plt.legend(fontsize=16,loc='upper right')

plt.subplot(222)
plt.grid()
for i in range(n):
    plt.plot(t,v[:,i],linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Speed (m/s)',fontsize=18)

plt.subplot(223)
plt.grid()
for i in range(n):
    plt.plot(t,a[:,i],linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Acceleration (m/s^2)',fontsize=18)

plt.subplot(224)
plt.grid()
for i in range(n):
    plt.plot(x[:,i],[potentials[0].potential(j,ensemble_prop) for j in x[:,i]],linewidth=2)
plt.xlabel('Position (m)',fontsize=18)
plt.ylabel('Potential energy (J)',fontsize=18)

plt.tight_layout()
plt.show()

### 2 IDENTICAL CHARGES COLOUMB POTENTIAL

Here we compare 2 identical charged atoms for their coloumb interaction

In [ ]:
# ENSEMBLE DEFINITIONS
"""
n       : The number of particles in the simulation
charge  : Charge of the particle in C
mass    : Mass of the particle in kg
"""
ensemble_prop = {
    "n" : 2,
    "charge" : np.array([1.0,1.0]),
    "mass": np.array([2.0,2.0])
}

n = ensemble_prop["n"]

# Initial position
x0 = np.array([-0.5 , 0.5])

# Initial velocity
v0 = np.array([0.0, 0.0])

# Eqlm position due to other potentials
d = np.array([0.0,0.0])

In [ ]:
# Creates local 1D harmonic potential around origin with frequency w
potentials = [InverseSquarePotential1D(d)]

In [ ]:
# SIMULATION SETUP

# Total time
T = 5

# Time step
dt = 1e-3

x,v,a = sim_leapfrog(T,dt,x0,v0,ensemble_prop,potentials,language='python',dims=1)
t = np.linspace(0,T,int(T/dt)+1)

In [ ]:
# PLOTTING
plt.figure(figsize=(13.33,7.5),dpi=96)

plt.grid()
for i in range(n):
    plt.plot(t,x[:,i],label='particle '+str(i+1),linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Position (m)',fontsize=18)
plt.legend(fontsize=18)
plt.rcParams['figure.figsize'] = [10/2.54, 8/2.54]
plt.show()

In [ ]:
plt.figure(figsize=(13.33,7.5),dpi=96)

plt.subplot(221)
plt.grid()
for i in range(n):
    plt.plot(t,x[:,i],label='particle '+str(i + 1),linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Position (m)',fontsize=18)
plt.legend(fontsize=16,loc='right')

plt.subplot(222)
plt.grid()
for i in range(n):
    plt.plot(t,v[:,i],linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Speed (m/s)',fontsize=18)

plt.subplot(223)
plt.grid()
for i in range(n):
    plt.plot(t,a[:,i],linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Acceleration (m/s^2)',fontsize=18)

plt.subplot(224)
plt.grid()
plt.plot(t,[potentials[0].potential(x[i,:],ensemble_prop) for i in range(len(t))],linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Potential energy (J)',fontsize=18)

plt.tight_layout()
plt.show()

### 2 DIFFERENT CHARGED PARTICLES

Here we compare 2 particles of different charges and masses for their coloumbic interactions

In [ ]:
# ENSEMBLE DEFINITIONS
"""
n       : The number of particles in the simulation
charge  : Charge of the particle in C
mass    : Mass of the particle in kg
"""
ensemble_prop = {
    "n" : 2,
    "charge" : np.array([1,3]),
    "mass": np.array([2,4])
}

n = ensemble_prop["n"]

# Initial position
x0 = np.array([-0.5, 0.5])

# Initial velocity
v0 = np.array([0.0, 0.0])

# Eqlm position due to other potentials
d = np.array([0.0,0.0])

In [ ]:
# Creates local 1D harmonic potential around origin with frequency w
potentials = [InverseSquarePotential1D(d)]

In [ ]:
# SIMULATION SETUP

# Total time
T = 5

# Time step
dt = 1e-3

x,v,a = sim_leapfrog(T,dt,x0,v0,ensemble_prop,potentials,language='c++',dims=1)
t = np.linspace(0,T,int(T/dt)+1)

In [ ]:
# PLOTTING
plt.figure(figsize=(13.33,7.5),dpi=96)

plt.grid()
for i in range(n):
    plt.plot(t,x[:,i],label='particle '+str(i+1),linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Position (m)',fontsize=18)
plt.legend(fontsize=18)
plt.rcParams['figure.figsize'] = [10/2.54, 8/2.54]
plt.show()

In [ ]:
plt.figure(figsize=(13.33,7.5),dpi=96)

plt.subplot(221)
plt.grid()
for i in range(n):
    plt.plot(t,x[:,i],label='particle '+str(i + 1),linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Position (m)',fontsize=18)
plt.legend(fontsize=16,loc='right')

plt.subplot(222)
plt.grid()
for i in range(n):
    plt.plot(t,v[:,i],linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Speed (m/s)',fontsize=18)

plt.subplot(223)
plt.grid()
for i in range(n):
    plt.plot(t,a[:,i],linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Acceleration (m/s^2)',fontsize=18)

plt.subplot(224)
plt.grid()
plt.plot(t,[potentials[0].potential(x[i,:],ensemble_prop) for i in range(len(t))],linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Potential energy (J)',fontsize=18)

plt.tight_layout()
plt.show()

### 2 CHARGED PARTICLES IN A HARMONIC POTENTIAL

2 charged particles moving in a harmonic potential

In [ ]:
# ENSEMBLE DEFINITIONS
"""
n       : The number of particles in the simulation
charge  : Charge of the particle in C
mass    : Mass of the particle in kg
"""
ensemble_prop = {
    "n" : 2,
    "charge" : np.array([1,3]),
    "mass": np.array([2,4])
}

n = ensemble_prop["n"]

# Initial position
x0 = np.array([-0.2 , 0.2])

# Initial velocity
v0 = np.array([0.0, 0.0])

# Angular frequency
w = np.array([np.sqrt(10.0),np.sqrt(5.0)])

# Eqlm position due to other potentials
d = np.array([0.0,0.0])

In [ ]:
# Creates local 1D harmonic potential around origin with frequency w
potentials = [InverseSquarePotential1D(d),LocalHarmonicPotential1D(w)]

In [ ]:
# SIMULATION SETUP

# Total time
T = 5

# Time step
dt = 1e-3

x,v,a = sim_leapfrog(T,dt,x0,v0,ensemble_prop,potentials,language='c++',dims=1)
t = np.linspace(0,T,int(T/dt)+1)

In [ ]:
# PLOTTING
plt.figure(figsize=(13.33,7.5),dpi=96)

plt.grid()
for i in range(n):
    plt.plot(t,x[:,i],label='particle '+str(i+1),linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Position (m)',fontsize=18)
plt.legend(fontsize=18)
plt.rcParams['figure.figsize'] = [10/2.54, 8/2.54]
plt.show()

In [ ]:
plt.figure(figsize=(13.33,7.5),dpi=96)

plt.subplot(221)
plt.grid()
for i in range(n):
    plt.plot(t,x[:,i],label='particle '+str(i + 1),linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Position (m)',fontsize=18)
plt.legend(fontsize=16,loc='right')

plt.subplot(222)
plt.grid()
for i in range(n):
    plt.plot(t,v[:,i],linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Speed (m/s)',fontsize=18)

plt.subplot(223)
plt.grid()
for i in range(n):
    plt.plot(t,a[:,i],linewidth=2)
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Acceleration (m/s^2)',fontsize=18)

plt.subplot(224)
plt.grid()
plt.plot(t,[potentials[0].potential(x[i,:],ensemble_prop) for i in range(len(t))],linewidth=2,label = "Coulombic potential")
for i in range(n):
    plt.plot(t,[potentials[1].potential(x[j][i],ensemble_prop) for j in range(len(t))],linewidth=2,label = "Harmonic pot : Particle " + str(i+1) )
plt.xlabel('Time (s)',fontsize=18)
plt.ylabel('Potential energy (J)',fontsize=18)
plt.legend()

plt.tight_layout()
plt.show()

**NOTE :** The coordinates for a multi-atom 3D system follows the pattern (x1, x2, x3 ... xn, y1, y2, y3 ... yn, z1, z2, z3 ... zn)

### 2 DIFFERENT CHARGED PARTICLES IN 3D

Here we compare 2 particles of different charges and masses for their coloumbic interactions in 3D

In [ ]:
# ENSEMBLE DEFINITIONS
"""
n       : The number of particles in the simulation
charge  : Charge of the particle in C
mass    : Mass of the particle in kg
"""

e = 1e-5

ensemble_prop = {
    "n" : 2,
    "charge" : np.array([e,3*e]),
    "mass": np.array([2,4])
}

n = ensemble_prop["n"]

# Initial position
x0 = np.array([-0.5, 0.5, -0.25, 0.25, -1.5, 1.5])

# Initial velocity
v0 = np.zeros(3*n)

# Cutoff
cutoff = None

In [ ]:
# Creates local 1D harmonic potential around origin with frequency w
potentials = [MutualCoulombPotential(cutoff)]

In [ ]:
# SIMULATION SETUP

# Total time
T = 3

# Time step
dt = 1e-3

x,v,a = sim_leapfrog(T,dt,x0,v0,ensemble_prop,potentials,language='c++',dims=3)
t = np.linspace(0,T,int(T/dt)+1)

In [ ]:
for ind,dimension in enumerate(["X","Y","Z"]):

    plt.figure(figsize=(13.33,7.5),dpi=96)

    plt.suptitle("Axis : "+dimension,fontsize = 18)
    plt.subplot(221)
    plt.grid()
    for i in range(n):
        plt.plot(t,x[:,ind*n + i],label='particle '+str(i + 1),linewidth=2)
    plt.xlabel('Time (s)',fontsize=18)
    plt.ylabel('Position (m)',fontsize=18)
    plt.legend(fontsize=16,loc='right')

    plt.subplot(222)
    plt.grid()
    for i in range(n):
        plt.plot(t,v[:,ind*n + i],linewidth=2)
    plt.xlabel('Time (s)',fontsize=18)
    plt.ylabel('Speed (m/s)',fontsize=18)

    plt.subplot(223)
    plt.grid()
    for i in range(n):
        plt.plot(t,a[:,ind*n + i],linewidth=2)
    plt.xlabel('Time (s)',fontsize=18)
    plt.ylabel('Acceleration (m/s^2)',fontsize=18)

    plt.subplot(224)
    plt.grid()
    plt.plot(t,[potentials[0].potential(x[i,:],ensemble_prop) for i in range(len(t))],linewidth=2)
    plt.xlabel('Time (s)',fontsize=18)
    plt.ylabel('Potential energy (J)',fontsize=18)

    plt.tight_layout()
    plt.show()

### TESTING FUNCTIONS

#### TESTING EXAMPLE POTENTIAL

This is a test plot for the `testPseudoPot()`. This also acts as an example for contour plots in the future.

In [ ]:
X,Y = np.mgrid[-1:1:100j,-1:1:100j]
Z = np.arange(-5,6,2)

A = 5
B = 4

obj = testPseudoPot(A,B)
fig = plt.figure(figsize=(13.33,13),dpi=96)

for idx,z in enumerate(Z):
    ax = fig.add_subplot(3,2,idx+1)
    
    cnt = ax.contour(X, Y, obj.contourPotential(X,Y,z), colors = "k", linewidths = 0.5)
    ax.clabel(cnt, cnt.levels, inline = True, fontsize = 10)

    cnt = ax.contourf(X,Y,obj.contourPotential(X,Y,z), cmap="viridis",vmin=-100,vmax=15)
    cbar = ax.figure.colorbar(cnt, ax = ax)
    cbar.ax.set_ylabel("Potential Energy",fontsize = 16, rotation = -90, va = "bottom")

    ax.set_title("Z = "+str(z),fontsize=16)
    ax.set_xlabel("X",fontsize=18)
    ax.set_ylabel("Y",fontsize=18)
    ax.tick_params(which="both",labelsize=12)

plt.tight_layout()
plt.show()

#### TESTING THE VELOCITY GENERATOR

Generating velovities at different temperatures and comparing with the actual distributions

In [ ]:
def MaxwellBoltzmann(v,m,T):
    kB = 1.38e-23
    return (m/(2*np.pi*kB*T))**1.5 * 4*np.pi * v*v * np.exp(-m*v*v/(2*kB*T))


In [ ]:
n = 100000
mBa = 2.28036828e-25
mass = np.ones(n)*mBa

v = np.arange(0,1000,0.1)
T = [100.0,200.0,300.0]

plt.figure(figsize=(13.33,7.5),dpi=96)

plt.grid()
for t in T:
    plt.plot(v,MaxwellBoltzmann(v,mBa,t),label='Temperature '+str(t) + 'K',linewidth=2)
plt.xlabel('Speed',fontsize=18)
plt.ylabel('P(v)',fontsize=18)
plt.legend(fontsize=18)
plt.rcParams['figure.figsize'] = [10/2.54, 8/2.54]
plt.show()

In [ ]:
colorList = ['blue','orange','green']
plt.figure(figsize=(13.33,7.5),dpi=96)

for idx,t in enumerate(T):
    velGenerated = genMaxwellVelocities(t,mass)
    speedGenerated = [velGenerated[i]*velGenerated[i] + velGenerated[i+n]*velGenerated[i+n] + velGenerated[i+2*n]*velGenerated[i+2*n] for i in range(n)]
    speedGenerated = np.array(speedGenerated)
    speedGenerated = np.sqrt(speedGenerated)
    plt.hist(speedGenerated,color=colorList[idx],alpha=0.4,bins=100, density=True)
    plt.plot(v,MaxwellBoltzmann(v,mBa,t),color=colorList[idx],label='Temperature '+str(t) + 'K',linewidth=2)

plt.xlabel('Speed',fontsize=18)
plt.ylabel('P(v)',fontsize=18)
plt.legend(fontsize=18)
plt.show()
    

### COLOUMB CRYSTAL
Attempt at an energy minimized structure of a coulom crystal

#### TESTING THE PSEUDO POTENTIAL

In [ ]:
# ENSEMBLE DEFINITIONS
"""
n       : The number of particles in the simulation
charge  : Charge of the particle in C
mass    : Mass of the particle in kg
"""

e = 1.6e-19
n = 100
mBa = 2.28036828e-26

ensemble_prop = {
    "n" : n,
    "charge" : e*np.ones(n),
    "mass": mBa*np.ones(n)
}

# Initial position
x0 = fibonacciSphere(n)

# Initial velocity
v0 = np.zeros(3*n)

#Alpha
alpha = -0.024

#Endcap voltage
Vend = 8

#RF voltage
Vrf = 160

#RF angular frequency
w = np.pi*1e6   #f = 500kHz

#Distance to electrode
r0 = 7.5e-3

# Cutoff
cutoff = None

In [ ]:
potentials = [LinearQuadrupolePseudoPotential(alpha,Vend,Vrf,w,r0,ensemble_prop["mass"]),MutualCoulombPotential(cutoff)]

In [ ]:
X,Y = np.mgrid[-5e-4:5e-4:100j,-5e-4:5e-4:100j]
Z_slice = np.linspace(-10,10,3)

fig = plt.figure(figsize=(12,9),dpi=96)
for idx,z in enumerate(Z_slice):
    ax = fig.add_subplot(2,2,idx+1)
    
    print("V(",z,")= ",potentials[0].contourPotential(0,0,z,ensemble_prop["mass"][0]))

    cnt = ax.contour(X,Y,potentials[0].contourPotential(X,Y,z,ensemble_prop["mass"][0]),linewidths = 2,cmap="viridis",vmin=0,vmax=1e-6,levels = 10)
    ax.clabel(cnt, cnt.levels, inline = True, fontsize = 10)

    ax.set_title('z = %.2f'%z, fontsize=18)
    ax.set_xlabel("X",fontsize=18)
    ax.set_ylabel("Y",fontsize=18)
    ax.tick_params(which="major",labelsize=12,rotation = -30)

plt.tight_layout()
plt.show()

In [ ]:
ax = plt.figure(figsize=(4,4),dpi=96).add_subplot(projection='3d')
ax.scatter3D(x0[:n],x0[n:2*n],x0[2*n:3*n])
plt.show()

In [ ]:
newpos = steepestDescent(x0,ensemble_prop,potentials,0.05,0.001,10000)

In [ ]:
ax = plt.figure(figsize=(4,4),dpi=96).add_subplot(projection='3d')
ax.scatter3D(newpos[:n],newpos[n:2*n],newpos[2*n:3*n])
plt.show()

In [ ]:
T = 1e-3
dt = 5e-6

r_sim, v_sim, a_sim = sim_leapfrog(T, dt, newpos, v0, ensemble_prop, potentials=potentials, language="python")
t_sim = np.linspace(0, T, int(T/dt)+1)

In [ ]:
print("A",potentials[0].a)
print("Q",potentials[0].q)

In [ ]:
t_sim = np.linspace(0, T, int(T/dt)+1)

In [ ]:
np.savez("PseudoPotential_10particle",t_sim,r_sim,v_sim,a_sim)